# 类代码编写细节

In [31]:
class Shareddata:
    data = 30

In [32]:
x = Shareddata()
y = Shareddata()

In [33]:
x.data, id(x), y.data, id(y), Shareddata.data, id(Shareddata.data)

In [34]:
x.data = 80
x.data, id(x), y.data, id(y), Shareddata.data, id(Shareddata.data)

对实例的属性进⾏赋值运算会在该实例内创建或修改名称，⽽不是在共享的类中

In [35]:
Shareddata.data = 99

In [37]:
x.data, id(x), y.data, id(y), Shareddata.data, id(Shareddata.data)

In [38]:
class SharedList:
    data = [1, 2, 3]

In [39]:
x = SharedList()
y = SharedList()
x.data, id(x), y.data, id(y), SharedList.data, id(SharedList.data)

In [40]:
x.data = [4, 5, 6]
y.data = [1, 2, 3]  # 赋值相同的值也会改变引用
x.data, id(x), y.data, id(y), SharedList.data, id(SharedList.data)

In [41]:
SharedList.data = ['abc']

In [42]:
x.data, id(x), y.data, id(y), SharedList.data, id(SharedList.data)

# 修改实例属性

In [43]:
class Mixname:
    data = 'spam'

    def __init__(self, name):
        self.data = name

    def display(self):
        print(self.data, Mixname.data)

In [44]:
x = Mixname('eggs')
y = Mixname('apple')
x.display(), y.display()

In [49]:
Mixname.display(x)

In [50]:
Mixname.display()

# 继承

In [51]:
class Super:
    def method(self):
        print('in Super.method')

In [54]:
class Sub(Super):
    def method(self):
        print('in Sub.method')
        Super.method(self)
        print('ending Super.method')

In [56]:
x = Super()
x.method()

In [57]:
x = Sub()
x.method()

In [58]:
class Super:

    def method(self):
        print('in Super.method')

    def delegate(self):
        self.action()


class Inheritor(Super):
    # Default behavior # Expected to be defined # Inherit method verbatim
    pass


class Replacer(Super):

    def method(self):
        # Replace method completely
        print('in Replacer.method')


class Extender(Super):

    def method(self):
        # Extend method behavior
        print('starting Extender.method')
        Super.method(self)
        print('ending Extender.method')


class Provider(Super):

    def action(self):
        # Fill in a required method
        print('in Provider.action')


In [60]:
for klass in (Inheritor, Replacer, Extender):
    print('\n' + klass.__name__ + '...')
    klass().method()
    print('\nProvider...')
    x = Provider()
    x.delegate()
    print('-' * 50)

In [61]:
class Super:

    def method(self):
        print('in Super.method')

    def delegate(self):
        self.action()

    def action(self):
        raise NotImplementedError('action must be defined')


In [63]:
x = Super()
x.delegate()

In [64]:
y = Provider()
y.delegate()

In [66]:
class Sub(Super):
    def action(self):
        print('in Sub.action')

In [67]:
z = Sub()
z.delegate()

# 抽象类

In [68]:
from abc import ABCMeta, abstractmethod


class Super(metaclass=ABCMeta):

    def delegate(self):
        self.action()

    @abstractmethod
    def action(self):
        pass

In [69]:
x = Super()

In [70]:
class Sub(Super):
    pass

In [71]:
y = Sub()

In [72]:
class Sub(Super):
    def action(self):
        print('in Sub.action')


In [74]:
z = Sub()
z.delegate()

## 命名空间

In [75]:
X = 11


def f():
    print(X)


def g():
    # Global (module) name/attribute (X, or manynames.X)
    # Access global X (11)
    X = 22
    # Local (function) variable (X, hides module X)
    print(X)


class C:
    X = 33

    def m(self):
        # Class attribute (C.X)
        X = 44
        self.X = 55

# Local variable in method (X) # Instance attribute (instance.X)

In [89]:
print(X)
f()
g()
print(X)
print('-' * 50)
obj = C()
print(obj.X)
obj.m()
print(obj.X)
print(C.X)

In [100]:
X = 1


def nester():
    print(X)

    class C:
        print(X)

        def method1(self):
            print(X)

        def method2(self):
            X = 3
            print(X)

    # Global: 1 # Global: 1 # Global: 1 # Hides global # Local: 3

    I = C()  # 1
    I.method1()  # 1
    I.method2()  # 3


print(X)  # 1
print('-' * 40)
nester()  # 1 1 1 3

In [106]:
X = 1


def nester():
    X = 2
    print(X)

    class C:
        X = 3
        print(X)

        def method1(self):
            print(X)  # 2 not 3
            print(self.X)  # 3

        def method2(self):
            X = 4
            print(X)  # 4
            self.X = 5
            print(self.X)  # 5

    I = C()  # 3
    I.method1()  # 2 3
    I.method2()  # 4 5


print(X)  # 1
print('-' * 40)
nester()  # 3 2 3 4 5

In [108]:
X = 1

def nester():
    X = 2  # Hides global 

    print(X)  # Local: 2

    class C:
        X = 3  # Class local hides nester's: C.X or I.X (not scoped) 
        print(X)  # Local: 3

        def method1(self):
            print(X)  # In enclosing def (not 3 in class!): 2
            print(self.X)  # Inherited class local: 3

        def method2(self):
            X = 4  # Hides enclosing (nester, not class)
            print(X)  # Local: 4
            self.X = 5  # Hides class
            print(self.X)  # Located in instance: 5

    I = C()
    I.method1()
    I.method2()


print(X)
print('-' * 40)
nester()

In [110]:
class Sup:
    def hello(self):
        self.data1 = 'spam'
        
class Sub(Sup):
    def hola(self):
        self.data2 = 'eggs'

In [111]:
x = Sub()

In [112]:
x.__dict__

In [114]:
x.__class__

In [115]:
Sub.__base__

In [125]:
Sup.__base__

In [119]:
y = Sub()
y.hello()
y.__dict__

In [121]:
y.hola()
y.__dict__

In [122]:
list(Sub.__dict__)

In [124]:
list(Sup.__dict__)

In [126]:
y.data1, y.__dict__['data1']

In [127]:
y.data3 = 'toast'
y.__dict__

In [128]:
y.__dict__['hello']

In [129]:
Sup.__base__

In [131]:
Sup.__base__.__base__